In [2]:
%env AWS_ACCESS_KEY_ID=projet-slums-sa
%env AWS_SECRET_ACCESS_KEY=hB2N6hCmp7JoFA6WHKT022WJ9lOc1oOr

In [3]:
import sys
sys.path.append('../src')
import shutil
import yaml
import py7zr
import os
import s3fs
from satellite_image import SatelliteImage
from utils import *
import matplotlib.pyplot as plt
import numpy as np
import rasterio.plot as rp
import re
from datetime import datetime

In [4]:
from utils import get_root_path

with open("../environment.yml") as f:
    environment = yaml.safe_load(f)

root_path = get_root_path()
bucket = environment["bucket"]
path_s3_cayenne_data = environment["sources"]["cayenne"]
path_local_cayenne_data = os.path.join(root_path, environment["local-path"]["cayenne"])

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
fs.get(f"{bucket}/{path_s3_cayenne_data}", f"{path_local_cayenne_data}")
with py7zr.SevenZipFile(f"{path_local_cayenne_data}", mode="r") as z:
    z.extractall()

shutil.move(
    os.path.splitext(os.path.basename(path_s3_cayenne_data))[0],
    os.path.splitext(path_local_cayenne_data)[0],
)


In [7]:
filename = '../data/Cayenne/16bits/ORT_2022072050325085_U22N/ORT_2022072050325085_0353_0545_U22N_16Bits.jp2'
date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d')

In [8]:
image = SatelliteImage.from_raster(
        filename,
        date = date, 
        n_bands = 4
    )

In [9]:
list_images = image.split(256)

In [6]:
list_images[0].normalize()

In [7]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(list_images[0].array, (1, 2, 0))[:,:,:3])
plt.xlabel('Pixels Colonnes')
plt.ylabel('Pixels Lignes')

In [10]:
from labeler import RILLabeler
from labeled_satellite_image import SegmentationLabeledSatelliteImage

labeler = RILLabeler(date)

In [19]:
import rasterio
rasterio.coords.BoundingBox(10,10,10,11)

In [11]:
mask = labeler.create_segmentation_label(image)

Warning to remove

In [15]:
image.bounds

In [11]:
image.normalize()

In [12]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask, alpha=0.3)

C'est le même résultat que dans le notebook de Clément

In [13]:
from labeled_satellite_image import DetectionLabeledSatelliteImage
label = labeler.create_detection_label(image)

In [14]:
label

In [15]:
len(label)

In [16]:
from shapely.geometry import box
import geopandas as gpd

boxes = []
for bounding_coordinates in label:
    boxes.append(box(*bounding_coordinates))
g = gpd.GeoSeries(boxes)
g

In [17]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
g.plot(color="red", ax=ax)

BDTOPO

In [19]:
from labeler import BDTOPOLabeler

labeler = BDTOPOLabeler(date)

In [21]:
mask = labeler.create_segmentation_label(image)

In [24]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask, alpha=0.3)

In [25]:
label = labeler.create_detection_label(image)

In [26]:
label

In [27]:
len(label)

In [28]:
from shapely.geometry import box
import geopandas as gpd

boxes = []
for bounding_coordinates in label:
    boxes.append(box(*bounding_coordinates))
g = gpd.GeoSeries(boxes)
g

In [29]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
g.plot(color="red", ax=ax)